Precisa instalar o Pygrib

In [1]:
import pandas as pd
import os
import numpy
import pygrib
import xarray as xr

In [2]:
# diretório onde estão os arquivos grib2
grib_dir = r"C:\IC\grib"

# salvando todos os arquivos grib2 em uma lista
grib_files = [os.path.join(grib_dir,pasta, file) for pasta, _, arquivos in os.walk(grib_dir) for file in arquivos if file.endswith('grib2')]

# for grb in arquivo:
# print(grb.latlons())
# print(grib_files)

# função para converter arquivos grib2 em netCDF
def convert_grib_to_netcdf(grib_files):

    # abrindo o arquivo grib2
    grbs = pygrib.open(grib_files)

    # criando um dataset vazio
    ds = xr.Dataset()

    # loop para pegar todas as variáveis do arquivo grib2 e salvar no dataset
    for grb in grbs:
        variable_name = grb.shortName
        data = grb.values
        lats, lons = grb.latlons()

        aux = xr.DataArray(data, coords=[lats[:,0], lons[0,:]], 
                            dims=['latitude', 'longitude'])

        aux.attrs['units'] = grb.units
        aux.attrs['long_name'] = grb.name
        ds[variable_name] = aux

    return ds

# lendo a quantidade de arquivos para calcular o progresso
quantidade_arquivos = len(grib_files)

# contador para registrar quantos arquivos foram lidos
arquivo_lido = 0

# loop para ler todos os arquivos na pasta
for arquivo_grib in grib_files:
    
    # pegando o nome do arquivo para salvar o netCDF
    nome_arquivo = os.path.basename(arquivo_grib).removesuffix('.grib2')
    
    # print(arquivo_grib)
    arquivo_netcdf = convert_grib_to_netcdf(arquivo_grib)

    # salvando arquivos netCDF em uma pasta
    arquivo_netcdf.to_netcdf(os.path.join(r"C:\IC\netCDF", f'{nome_arquivo}.nc'))

    # contador para registrar quantos arquivos foram lidos e calculando o progresso
    arquivo_lido += 1

    # printando o progresso
    print(f'\r{arquivo_lido*100/quantidade_arquivos:.2f}%',end = '\r')
